Make sure the zip codes have not been created since the year 2000.

In [28]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [29]:
#create list of zipcodes from txt file with zips
zips = open('chosen_zips_2000.txt', 'r').readlines()
zips = [z.replace('\n', '') for z in zips]
zipcode = zips


In [30]:
df_main = pd.DataFrame()
i = 0
num = len(zipcode)
for i in range(num):
    
    baseURL = 'https://www.zip-codes.com/zip-code/'+zipcode[i]+'/zip-code-'+zipcode[i]+'-2000-census.asp'


    #calledURL = baseURL % (zipcode, zipcode)
    source = requests.get(baseURL)
    soup = BeautifulSoup(source.text, 'lxml')

    #get data by looking for each tr
    #turns it all into a list, and we're using list comprehension
    data = []
    for tr in soup.find_all('tr'):
        values = [td.text for td in tr.find_all('td')]
        data.append(values)
        #the info we want is within a list of lists
        #just searching for one part of the sublist will not work
        

    #there has to be a better way to do this
    #but for now I have to create a sublist for every search column
    basic = [[],[],[],[],[],[],[],[],[],[]]
    search = ['Total 2000 Census Population for  ZIP Code '+zipcode[i],
          'Total households',
          'Average household size', 
          'Median age (years)',
          '18 years and over',
          '65 years and over',
         ]


    i = 0
    num = len(search)
    for i in range(num):
        for sublist in data:
            if sublist[0] == search[i]:
                basic[i].append(sublist[1])
        i = i +1


    #because we only need the 0 element in each sublist, drop the rest
    #this gives us the 2000 values for this zipcode
    df = pd.DataFrame(data=basic)
    df_tr = df.transpose()
    df_tr = df_tr[:1]
    

    df_main = df_main.append(df_tr)



#now lets add the zip code and year columns
df = df_main
df['year'] = '2000'
df['zipcode'] = zipcode


In [31]:
#drop the empty columns
df = df.drop(df.columns[[6,7,8,9]], axis = 1)

df.columns = ['pop', 'hhold', 'avg_hhold', 'med_age', '18plus', '65plus', 'year', 'zipcode']

#join the town names
name_df = pd.read_excel("town_names.xlsx", dtype={'zipcode': object})
df = df.set_index('zipcode').join(name_df.set_index('zipcode'))



#move some columns around so it will be easier to read in excel
#remember zipcode is the index not a column
df = df[['territory', 'town', 'county', 'year', 'pop', 
         'hhold', 'avg_hhold', 'med_age', '18plus', '65plus' ]]

In [38]:
df.head()

,territory,town,county,year,pop,hhold,avg_hhold,med_age,18plus,65plus
zipcode,,,,,,,,,,
07059,Warren,Warren,Somerset,2000,14219,4616,3.05,39.1,9994,1592
07060,Warren,Plainfield,Union,2000,49040,16161,2.96,32.4,36317,4868
07062,Warren,Plainfield,Union,2000,13303,4561,2.91,33.2,9773,1362
07063,Warren,Plainfield,Union,2000,12153,3698,3.28,31.6,8592,1081
07076,NaN,Scotch Plains,Union,2000,19584,7216,2.69,37.8,14689,2794


Notice how there are thousand seperator comma's above, this can be a pain later if we don't remove them now

In [37]:
df['pop'] = df['pop'].str.replace(',','').astype(int)
df['hhold'] = df['hhold'].str.replace(',','').astype(int)
df['18plus'] = df['18plus'].str.replace(',','').astype(int)
df['65plus'] = df['65plus'].str.replace(',','').astype(int)

In [39]:
df.to_excel('./excel/2000_scrape.xlsx')